In [6]:
import OpenDartReader
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
pd.set_option('display.max_columns', 20)

In [14]:
my_api = open("dart_api.txt").read()[1:]
dart = OpenDartReader(my_api)

ValueError: {'status': '020', 'message': '사용한도를 초과하였습니다.'}

In [15]:
def find_div_and_EPS(stock, year):
    report = dart.report(stock, "배당", year, "11011")
    output = dict()

    if report is None:  # 리포트가 없으면 None을 반환하므로 더미값 삽입
        output["주당배당금"] = np.nan, np.nan, np.nan
        output["주당순이익"] = np.nan, np.nan, np.nan
    else:
        # 주당배당금 관련 행 필터링
        div_row = report.loc[(report['se'] == '주당 현금배당금(원)')].iloc[0]

        # 배당 기록이 없으면 '-' 으로 나타남
        cur_div = int(div_row["thstrm"].replace("-", "0").replace(",", ""))  # 당기(current)
        pre_div = int(div_row["frmtrm"].replace("-", "0").replace(",", ""))  # 전기(previous)
        spre_div = int(div_row["lwfr"].replace("-", "0").replace(",", ""))  # 전전기(second previous)
        output["주당배당금"] = spre_div, pre_div, cur_div

        # 주당순이익 관련 행 필터링
        EPS_row = report.loc[(report['se'].str.contains('주당순이익'))].iloc[0]

        cur_EPS = int(EPS_row['thstrm'].replace('-', '0').replace(',', ''))
        pre_EPS = int(EPS_row['frmtrm'].replace('-', '0').replace(',', ''))
        spre_EPS = int(EPS_row['lwfr'].replace('-', '0').replace(',', ''))
        output["주당순이익"] = spre_EPS, pre_EPS, cur_EPS

    return output

In [16]:
stock_list = pd.read_csv("data/종목정보.txt", sep="\t", encoding="euc-kr")
stock_name_list = stock_list["Name"].values
div_data = []
EPS_data = []

In [17]:
for idx, stock_name in tqdm(enumerate(stock_name_list)):
    print(idx + 1, "/", len(stock_name_list))

    # 레코드 초기화
    div_record = [stock_name]
    EPS_record = [stock_name]

    for year in [2015, 2018, 2020]:  # 한번당 3년을 가져오기 때문에 간격을 고려해서 루프
        while True:
            try:
                output = find_div_and_EPS(stock_name, year)
                time.sleep(0.5)
                break
            except:
                time.sleep(10)

        # 주당 배당금 정리
        spre_divs, pre_divs, cur_divs = output["주당배당금"]
        if year != 2020:
            div_record += [spre_divs, pre_divs, cur_divs]
        else:
            div_record += [pre_divs, cur_divs]  # 2020년이 되면 18년이 중복되므로 제외

        # 주당 순이익 정리
        spre_EPS, pre_EPS, cur_EPS = output["주당순이익"]
        if year != 2020:
            div_record += [spre_EPS, pre_EPS, cur_EPS]
        else:
            div_record += [pre_EPS, cur_EPS]

    div_data.append(div_record)
    EPS_data.append(EPS_record)

0it [00:00, ?it/s]

1 / 2370


0it [00:10, ?it/s]


KeyboardInterrupt: 

In [18]:
# 데이터프레임으로 변환 및 저장
columns = ["stock_name", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"]
div_data = pd.DataFrame(div_data, columns=columns)
EPS_data = pd.DataFrame(EPS_data, columns=columns)

div_data.to_csv("data/주당배당금.csv", encoding="euc-kr", index=False)
EPS_data.to_csv("data/주당순이익.csv", encoding="euc-kr", index=False)

In [19]:
# 제출 마감일 계산
# 사업보고서 제출 시점은 사업연도 경과 후 기준일 이후 90일째 되는 날
# ex) 결산 월이 12월인 기업의 사업연도 경과 후 기준일은 다음 날인 1월 1일이고 90일째 되는 날은 3월 31일. 이때 당일이 공휴일이라면(2019년은 당일이 일요일) 가장 가까운 다음 평일이 제출 시점이 됨

sub_due_data = []  # 제출 마감일을 담을 데이터

for settle_month in range(1, 13):  # 결산 월
    for year in range(2013, 2021):  # 사업연도
        # 사업연도 경과 후 기준일 계산
        if settle_month < 12:
            after_bs_year_day = pd.to_datetime("{}-{}-1".format(year, settle_month+1))
        else:
            after_bs_year_day = pd.to_datetime("{}-1-1".format(year+1))

        due_date = after_bs_year_day + pd.to_timedelta(90, "D")
        sub_due_data.append([settle_month, year, due_date])

sub_due_data = pd.DataFrame(sub_due_data, columns=["결산 월", "사업연도", "제출마감일"])
sub_due_data.to_csv("data/사업보고서_제출마감일.csv", index=False, encoding="euc-kr")

In [20]:
# 장이 열리는 가장 가까운 날짜의 주가 찾기
# 주가 데이터와 날짜가 입력됐을 때 해당 날짜를 포함해서 가장 가까운 미래의 주가를 반환하는 함수

def find_closest_stock_price(sp_data, date):
    date = pd.to_datetime(date)

    # 주가 데이터를 벗어나는 범위의 날짜가 입력되면 결측을 반환
    if sp_data["Date"].max() < date:
        return np.nan
    else:
        while True:
            # date와 같은 날짜가 Date에 있으면 해당 날짜의 종가를 저장
            if sum(sp_data["Date"] == date) > 0:
                value = sp_data.loc[sp_data["Date"] == date, "Close"].iloc[0]
                break
            else:  # date와 같은 날짜의 Date가 없으면 당일에 장이 열리지 않은것이므로 date에 1을 더해서 다음날 장이 열렸는지 확인
                date += pd.to_timedelta(1, 'D')
        return value

In [21]:
# 정의한 함수를 삼성전자의 2020년 4월 4일로 확인. 이날은 토요일

sp_data = pd.read_csv("data/주가데이터/삼성전자.csv")
sp_data["Date"] = pd.to_datetime(sp_data["Date"])
date = "2020-04-04"
print(find_closest_stock_price(sp_data, date))  # 4월 6일의 종가인 48700을 올바르게 반환

48700


In [22]:
# 주가 데이터를 EPS_data와 동일하게 변환

settle_month_dict = stock_list.set_index("Name")["SettleMonth"]
settle_month_dict = settle_month_dict.apply(lambda  x:int(x[:-1])).to_dict()
sub_due_dict = sub_due_data.set_index(["결산 월", "사업연도"])["제출마감일"].to_dict()

In [25]:
print(settle_month_dict["삼성전자"])
print(sub_due_dict[12, 2017])

# 삼성전자의 결산 월은 12월이고, 2017년 12월의 제출마감일은 2018년 4월 1일 00시

12
2018-04-01 00:00:00


In [32]:
# EPS_data의 stock_name에 속한 모든 종목의 연도별 사업보고서 제출 마감일의 종가 저장

import os
sp_data = []

EPS_data = pd.read_csv("data/주당배당금.csv", encoding="euc-kr")

for sn in tqdm(EPS_data["stock_name"].values):
    record = [sn]
    if sn + ".csv" not in os.listdir("data/주가데이터"):
        # 주가데이터 폴더 내에 해당 파일이 없으면 전부 결측으로 채움
        record += [np.nan] * (len(EPS_data.columns) - 1)
    else:
        # 주가 데이터 불러오기
        sn_sp_data = pd.read_csv("data/주가데이터/{}.csv".format(sn), parse_dates=["Date"])
        settle_month = settle_month_dict[sn]

        for year in range(2013, 2021):
            # 제출 마감일의 주가 찾기
            sub_date = sub_due_dict[settle_month, year]
            sp = find_closest_stock_price(sn_sp_data, sub_date)
            record.append(sp)

    sp_data.append(record)
sp_data = pd.DataFrame(sp_data, columns=EPS_data.columns)

100%|██████████| 2370/2370 [11:19<00:00,  3.49it/s]


In [33]:
print(sp_data.head())

  stock_name     2013     2014     2015     2016     2017     2018     2019  \
0         3S   6100.0   4350.0   2535.0   3595.0   2320.0   2685.0   2700.0   
1     AJ네트웍스   7100.0   7100.0   8300.0   5800.0   7170.0   4810.0   2950.0   
2      AK홀딩스  55731.0  98500.0  56100.0  61200.0  74600.0  52400.0  18000.0   
3     APS홀딩스   6504.0   6929.0  13672.0  20750.0   7210.0   4420.0   6450.0   
4      AP시스템  41700.0  41700.0  41700.0  41700.0  27150.0  28800.0  24500.0   

      2020  
0   2420.0  
1   4620.0  
2  26900.0  
3   9800.0  
4  28800.0  


In [35]:
# sp_data를 EPS_data로 나누어 PER 계산
# PER = 주가/EPS
# EPS가 0일땐 계산이 되지 않으므로 nan으로 대체. nan은 연산해도 nan
PER_data = sp_data.values[:, 1:] / EPS_data.replace(0, np.nan).values[:, 1:]
PER_data = pd.DataFrame(PER_data, columns=EPS_data.columns[1:])
PER_data["stock_name"] = EPS_data["stock_name"]

In [36]:
PER_data.head()

,2013,2014,2015,2016,2017,2018,2019,2020,stock_name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3S
1,NaN,NaN,NaN,96.666667,83.372093,48.1,9.833333,22.0,AJ네트웍스
2,278.655,281.428571,112.2,111.272727,114.769231,69.866667,24.0,67.25,AK홀딩스
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,APS홀딩스
4,NaN,NaN,NaN,NaN,NaN,192.0,490.0,240.0,AP시스템


In [37]:
PER_data.to_csv("data/PER.csv", encoding="euc-kr", index=False)